# Model Training

### Import libraries

In [1]:
# mass imports
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

Read in processed file and labels file then combine them.

In [2]:
data = pd.read_csv("./data/final_processed.csv")
labels = pd.read_csv("./data/final_labels.csv")
combined = pd.concat([data, labels.label], axis=1)
combined.head()

,bookingID,Accuracy,Speed,acceleration,gyro,second,label
0,0,10.127617,22.946083,12.988328,0.749086,1589.0,0
1,1,3.706486,21.882141,12.790147,0.717864,1034.0,1
2,2,3.930626,9.360483,13.403410,0.463685,825.0,1
3,4,10.000000,19.780001,21.053265,0.661675,1094.0,1
4,6,4.586721,16.394695,14.498268,0.626294,1094.0,0


Separate training features and label.

In [3]:
x = combined.drop(["label","bookingID"], axis=1)
y = combined[['label']]

Split dataset into 80% for training, 20% for testing.

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, test_size = 0.2, random_state = 1)

SyntaxError: unexpected EOF while parsing (<ipython-input-4-d9f4ea5bb887>, line 1)

Set the parameters for the XgBoost classifier.

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=250,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=1,
 colsample_bytree=1,
 objective= 'binary:logistic',
 n_jobs=4,
 scale_pos_weight=3,
 random_state=1)

Feed the training data into the classifier and save the model.

In [ ]:
x_train = x_train.values
x_test = x_test.values
xgb1 = xgb1.fit(x_train,y_train.values.ravel())
pickle.dump(xgb1, open("./models/xgboost_model", "wb"), protocol=4)

Check the accuracy and f1_score for the prediction results and view its confusion matrix.

In [ ]:
y_pred = xgb1.fit(x_train,y_train.values.ravel()).predict(x_test)
scores = cross_val_score(xgb1, x_train, y_train.values.ravel(), cv=10, scoring = "roc_auc")
print(scores.mean())
print(f1_score(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)